# Whole Brain Analysis: GLM first level analysis

## Import

In [ ]:
import pandas as pd
import numpy as np
import os 

from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker

import glob
from natsort import natsorted

import matplotlib.pyplot as plt

## Standardize by section

In [ ]:
lstm_rnng_regressors = pd.read_csv('../regressors/BCCWJ_regressors.tsv',sep= '\t')
lstm_rnng_regressors['constant'] = np.ones(len(lstm_rnng_regressors))

lstm_rnng_regressors1 = lstm_rnng_regressors[lstm_rnng_regressors.section_number==1].drop(columns=['section_number'])
lstm_rnng_regressors1 = (lstm_rnng_regressors1-lstm_rnng_regressors1.mean())/lstm_rnng_regressors1.std()
lstm_rnng_regressors1['constant'] = np.ones(len(lstm_rnng_regressors1))

lstm_rnng_regressors2 = lstm_rnng_regressors[lstm_rnng_regressors.section_number==2].drop(columns=['section_number'])
lstm_rnng_regressors2 = (lstm_rnng_regressors2-lstm_rnng_regressors2.mean())/lstm_rnng_regressors2.std()
lstm_rnng_regressors2['constant'] = np.ones(len(lstm_rnng_regressors2))

lstm_rnng_regressors3 = lstm_rnng_regressors[lstm_rnng_regressors.section_number==3].drop(columns=['section_number'])
lstm_rnng_regressors3 = (lstm_rnng_regressors3-lstm_rnng_regressors3.mean())/lstm_rnng_regressors3.std()
lstm_rnng_regressors3['constant'] = np.ones(len(lstm_rnng_regressors3))

lstm_rnng_regressors4 = lstm_rnng_regressors[lstm_rnng_regressors.section_number==4].drop(columns=['section_number'])
lstm_rnng_regressors4 = (lstm_rnng_regressors4-lstm_rnng_regressors3.mean())/lstm_rnng_regressors4.std()
lstm_rnng_regressors4['constant'] = np.ones(len(lstm_rnng_regressors4))

lstm_rnng_design_matrices = [lstm_rnng_regressors1,lstm_rnng_regressors2,lstm_rnng_regressors3,lstm_rnng_regressors4]
for dmx in lstm_rnng_design_matrices:
    print(len(dmx))

## The design matrix

In [ ]:
plotting.plot_design_matrix(lstm_rnng_design_matrices[0])
plt.savefig('design_matrices.png')

## First -Level GLM models

In [ ]:
bccwj_subjects = natsorted(glob.glob("../data/nii_wb/*"))
bccwj_subjects

In [ ]:
natsorted(glob.glob(f"{bccwj_subjects[0]}/*"))

In [ ]:
path_list = ['../data/glm','../data/glm/word_rate', '../data/glm/word_length','../data/glm/word_freq', '../data/glm/sent_id', '../data/glm/sent_pos', '../data/glm/ngram', '../data/glm/lstm', '../data/glm/surp_rnng_td', '../data/glm/surp_rnng_lc', '../data/glm/dis_rnng_td', '../data/glm/dis_rnng_lc']
for path in path_list:
    os.makedirs(path,exist_ok=True)

In [ ]:
for i in range(len(bccwj_subjects)):
#for i in range(1):
    try:
        print(bccwj_subjects[i])
        
        bccwj_glm = FirstLevelModel(
                    t_r = 2.0,
                    hrf_model='spm',
                    noise_model='ar1',
                    smoothing_fwhm=None,
                    n_jobs=1,
                    standardize=True,
                    verbose=1
                    )
        bccwj_glm = bccwj_glm.fit(natsorted(glob.glob(f"{bccwj_subjects[i]}**/*")), design_matrices=lstm_rnng_design_matrices)
        
        word_rate_eff_map = bccwj_glm.compute_contrast(contrast_def='word_rate',output_type='effect_size')
        word_rate_eff_map.to_filename(f"../data/glm/word_rate/subj{i}_word_rate.nii")  
        
        word_length_eff_map = bccwj_glm.compute_contrast(contrast_def='word_length',output_type='effect_size')
        word_length_eff_map.to_filename(f"../data/glm/word_length/subj{i}_word_length.nii") 
        
        word_freq_eff_map = bccwj_glm.compute_contrast(contrast_def='word_freq',output_type='effect_size')
        word_freq_eff_map.to_filename(f"../data/glm/word_freq/subj{i}_word_freq.nii") 
        
        sent_id_eff_map = bccwj_glm.compute_contrast(contrast_def='sentid',output_type='effect_size')
        sent_id_eff_map.to_filename(f"../data/glm/sent_id/subj{i}_sent_id.nii")
        
        sent_pos_eff_map = bccwj_glm.compute_contrast(contrast_def='sentpos',output_type='effect_size')
        sent_pos_eff_map.to_filename(f"../data/glm/sent_pos/subj{i}_sentpos.nii")
        
        surp_ngram_five_eff_map = bccwj_glm.compute_contrast(contrast_def='surp.ngram_five',output_type='effect_size')
        surp_ngram_five_eff_map.to_filename(f"../data/glm/ngram/subj{i}_ngram.nii")
        
        surp_lstm_eff_map = bccwj_glm.compute_contrast(contrast_def='surp.LSTM',output_type='effect_size')
        surp_lstm_eff_map.to_filename(f"../data/glm/lstm/subj{i}_lstm.nii")
        
        surp_rnng_td_eff_map = bccwj_glm.compute_contrast(contrast_def='surp.RNNG_TD',output_type='effect_size')
        surp_rnng_td_eff_map.to_filename(f"../data/glm/surp_rnng_td/subj{i}_surp_rnng_td.nii")
        
        surp_rnng_lc_eff_map = bccwj_glm.compute_contrast(contrast_def='surp.RNNG_LC',output_type='effect_size')
        surp_rnng_lc_eff_map.to_filename(f"../data/glm/surp_rnng_lc/subj{i}_surp_rnng_lc.nii")
        
        dis_rnng_td_eff_map = bccwj_glm.compute_contrast(contrast_def='dis_RNNG_TD',output_type='effect_size')
        dis_rnng_td_eff_map.to_filename(f"../data/glm/dis_rnng_td/subj{i}_dis_rnng_td.nii")
        
        dis_rnng_lc_eff_map = bccwj_glm.compute_contrast(contrast_def='dis_RNNG_LC',output_type='effect_size')
        dis_rnng_lc_eff_map.to_filename(f"../data/glm/dis_rnng_lc/subj{i}_dis_rnng_lc.nii")
        
    except:
        print("something went wrong")  